***
some experiments with roi geometries: selection of the reference point to create congruent roi's
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils



***
__selection of reference point__

assume reference image\
assume roi to be specified in (an __integer__) number-of-pixels (size) in context of this image or projection

to obtain a symetrical roi, in this image (raster), around a (reference) point, :
- in case of an odd size, this reference point should be the center of a pixel
- in case of an even size, this reference point should be on a pixels border (intersection on grid)
***

__Proba V 333m__

In [ ]:
eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.somePV333ndviImageNear(eedate, eepoint)

eerefcenterpoint = geeutils.pixelcenterpoint(eepoint,eeimage)
eerefborderpoint = geeutils.pixelinterspoint(eepoint,eeimage)
#print(geeutils.szgeometryinfo(eerefcenterpoint))
#print(geeutils.szgeometryinfo(eerefborderpoint))

roisizes = [ 1,2,3,4,5,6 ]
    
map = geemap.Map(height='500px')
map.centerObject(eepoint, 14)
map.addLayer(eeimage, {'min':0, 'max':1})
for roisize in roisizes:
    size = round(roisize) #  "an integer" I said.
    if (size %2) == 0:
        # even
        roi = geeutils.squarerasterboundsroi(eerefborderpoint, size/2, eeimage) # pixelsradius will be 1, 2, ...
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 2), {'palette':'#0000ff', 'opacity':0.9})
    else:
        # odd
        roi = geeutils.squarerasterboundsroi(eerefcenterpoint, size/2, eeimage) # pixelsradius will be 0.5, 1.5, ...
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 7), {'palette':'#ff0000', 'opacity':0.3})
    #print(geeutils.szgeometryinfo(roi))
map.addLayer(geeutils.outlinegeometryimage(eepoint,          30, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eerefcenterpoint, 30, 2), {'palette':'#ff0000'}, 'eerefcenterpoint')
map.addLayer(geeutils.outlinegeometryimage(eerefborderpoint, 30, 2), {'palette':'#0000ff'}, 'eerefborderpoint')
map


__Sentinel 1 VV/VH bands__

In [ ]:
eepoint = geeutils.bobspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someS1rviImageNear(eedate, eepoint)

eerefcenterpoint = geeutils.pixelcenterpoint(eepoint,eeimage)
eerefborderpoint = geeutils.pixelinterspoint(eepoint,eeimage)

roisizes = [ 1,2,3,4,9,10 ]
    
map = geemap.Map(height='500px')
map.centerObject(eepoint, 18)
map.addLayer(eeimage, {'min':0, 'max':1})
for roisize in roisizes:
    size = round(roisize)  # btw: did you know round(1.5) == 2 == round(2.5)? neat...
    if (size %2) == 0:
        # even
        roi = geeutils.squarerasterboundsroi(eerefborderpoint, size/2, eeimage)
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 2), {'palette':'#0000ff', 'opacity':0.9})
    else:
        # odd
        roi = geeutils.squarerasterboundsroi(eerefcenterpoint, size/2, eeimage)
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 7), {'palette':'#ff0000', 'opacity':0.3})
    #print(geeutils.szgeometryinfo(roi))
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eerefcenterpoint, 1, 2), {'palette':'#ff0000'}, 'eerefcenterpoint')
map.addLayer(geeutils.outlinegeometryimage(eerefborderpoint, 1, 2), {'palette':'#0000ff'}, 'eerefborderpoint')
map


__Sentinel 1 'angle' band__
- compare results for COPERNICUS/S1_GRD and (undocumented?) COPERNICUS/S1_GRD_**FLOAT** collection


In [ ]:
if False:
    collection = geeutils.s1rbgImageCollection
else:
    s1floatImageCollection    = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
    s1floatrbgImageCollection = (s1floatImageCollection
                               .filter(ee.Filter.listContains('system:band_names','VV'))
                               .filter(ee.Filter.listContains('system:band_names','VH'))
                               .filter(ee.Filter.listContains('system:band_names','angle'))
                               .select(['VV', 'VH', 'angle']))                                  # False Color - not everywhere available
    collection = s1floatrbgImageCollection

eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someImageNear(collection.select('angle'), eedate, eepoint)

min = eeimage.reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
max = eeimage.reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

eerefcenterpoint = geeutils.pixelcenterpoint(eepoint,eeimage)
eerefborderpoint = geeutils.pixelinterspoint(eepoint,eeimage)

print(geeutils.szestimatevaluesinfo(eeimage.clip(eepoint.buffer(100000))))

roisizes = [ 1,2,3,4,5,6 ]
    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 8)
map.addLayer(eeimage, {'min':min, 'max':max})
for roisize in roisizes:
    size = round(roisize)
    if (size %2) == 0:
        # even
        roi = geeutils.squarerasterboundsroi(eerefborderpoint, size/2, eeimage)
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 2), {'palette':'#0000ff', 'opacity':0.9})
    else:
        # odd
        roi = geeutils.squarerasterboundsroi(eerefcenterpoint, size/2, eeimage)
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 7), {'palette':'#ff0000', 'opacity':0.3})
    #print(geeutils.szgeometryinfo(roi))
map.addLayer(geeutils.outlinegeometryimage(eepoint,          800, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eerefcenterpoint, 800, 2), {'palette':'#ff0000'}, 'eerefcenterpoint')
map.addLayer(geeutils.outlinegeometryimage(eerefborderpoint, 800, 2), {'palette':'#0000ff'}, 'eerefborderpoint')
map

__Sentinel 2 radio bands__

In [ ]:
eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someS2ndviImageNear(eedate, eepoint)

eerefcenterpoint = geeutils.pixelcenterpoint(eepoint,eeimage)
eerefborderpoint = geeutils.pixelinterspoint(eepoint,eeimage)

roisizes = [ 1,2,3,4,9,10 ]
    
map = geemap.Map(height='500px')
map.centerObject(eepoint, 18)
map.addLayer(eeimage, {'min':0, 'max':1})
for roisize in roisizes:
    size = round(roisize)
    if (size %2) == 0:
        # even
        roi = geeutils.squarerasterboundsroi(eerefborderpoint, size/2, eeimage)
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 2), {'palette':'#0000ff', 'opacity':0.9})
    else:
        # odd
        roi = geeutils.squarerasterboundsroi(eerefcenterpoint, size/2, eeimage)
        map.addLayer(geeutils.outlinegeometryimage(roi, 0, 7), {'palette':'#ff0000', 'opacity':0.3})
    #print(geeutils.szgeometryinfo(roi))
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eerefcenterpoint, 1, 2), {'palette':'#ff0000'}, 'eerefcenterpoint')
map.addLayer(geeutils.outlinegeometryimage(eerefborderpoint, 1, 2), {'palette':'#0000ff'}, 'eerefborderpoint')
map
